In [1]:
pip install yfinance

In [2]:
import yfinance as yf

def fetch_stock_data(stock_symbol, start_date, end_date):
    stock = yf.Ticker(stock_symbol)
    data = stock.history(start=start_date, end=end_date)
    return data

# 示例：获取苹果公司(AAPL)在指定日期范围内的股票数据
data = fetch_stock_data('AAPL', '2023-01-01', '2023-01-10')
print(data)


/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


                                 Open        High         Low       Close  \
Date                                                                        
2023-01-03 00:00:00-05:00  129.555841  130.172390  123.479803  124.374802   
2023-01-04 00:00:00-05:00  126.184691  127.944857  124.384755  125.657639   
2023-01-05 00:00:00-05:00  126.423345  127.059787  124.066524  124.325073   
2023-01-06 00:00:00-05:00  125.309579  129.565780  124.195801  128.899506   
2023-01-09 00:00:00-05:00  129.744803  132.668464  129.168025  129.426575   

                              Volume  Dividends  Stock Splits  
Date                                                           
2023-01-03 00:00:00-05:00  112117500        0.0           0.0  
2023-01-04 00:00:00-05:00   89113600        0.0           0.0  
2023-01-05 00:00:00-05:00   80962700        0.0           0.0  
2023-01-06 00:00:00-05:00   87754700        0.0           0.0  
2023-01-09 00:00:00-05:00   70790800        0.0           0.0  


In [ ]:
import sqlite3
import yfinance as yf

def check_stock_validity(stock_symbol):
    stock = yf.Ticker(stock_symbol)
    # 检查股票数据是否存在
    return stock.info['regularMarketPrice'] is not None

def add_stock_to_portfolio(portfolio_id, stock_symbol):
    if check_stock_validity(stock_symbol):
        try:
            conn = sqlite3.connect('stock_portfolio.db')
            cursor = conn.cursor()

            # 在这里，我们默认股票名称为股票代码
            cursor.execute("INSERT INTO stocks (portfolio_id, stock_symbol, stock_name) VALUES (?, ?, ?)",
                           (portfolio_id, stock_symbol, stock_symbol))
            conn.commit()
            print("股票添加成功")
        except sqlite3.Error as e:
            print(f"数据库错误: {e}")
        finally:
            conn.close()
    else:
        print("无效的股票代码")


In [ ]:
def remove_stock_from_portfolio(portfolio_id, stock_symbol):
    try:
        conn = sqlite3.connect('stock_portfolio.db')
        cursor = conn.cursor()

        cursor.execute("DELETE FROM stocks WHERE portfolio_id = ? AND stock_symbol = ?", (portfolio_id, stock_symbol))
        conn.commit()

        if cursor.rowcount > 0:
            print("股票移除成功")
        else:
            print("未找到该股票")
    except sqlite3.Error as e:
        print(f"数据库错误: {e}")
    finally:
        conn.close()


In [ ]:
def display_portfolios():
    try:
        conn = sqlite3.connect('stock_portfolio.db')
        cursor = conn.cursor()

        cursor.execute("SELECT * FROM portfolios")
        portfolios = cursor.fetchall()

        for portfolio in portfolios:
            print(f"组合ID: {portfolio[0]}, 创建日期: {portfolio[1]}")
            cursor.execute("SELECT stock_symbol FROM stocks WHERE portfolio_id = ?", (portfolio[0],))
            stocks = cursor.fetchall()
            print("包含股票: " + ", ".join([stock[0] for stock in stocks]))
            print()

    except sqlite3.Error as e:
        print(f"数据库错误: {e}")
    finally:
        conn.close()


In [3]:
import sqlite3
import yfinance as yf

# 检查股票是否有效
def check_stock_validity(stock_symbol):
    stock = yf.Ticker(stock_symbol)
    try:
        # 检查股票数据是否存在
        return stock.info['regularMarketPrice'] is not None
    except KeyError:
        return False

# 向组合添加股票
def add_stock_to_portfolio(portfolio_id, stock_symbol):
    if check_stock_validity(stock_symbol):
        try:
            conn = sqlite3.connect('stock_portfolio.db')
            cursor = conn.cursor()

            cursor.execute("INSERT INTO stocks (portfolio_id, stock_symbol, stock_name) VALUES (?, ?, ?)",
                           (portfolio_id, stock_symbol, stock_symbol))
            conn.commit()
            print(f"股票 {stock_symbol} 添加成功")
        except sqlite3.Error as e:
            print(f"数据库错误: {e}")
        finally:
            conn.close()
    else:
        print(f"无效的股票代码: {stock_symbol}")

# 从组合中移除股票
def remove_stock_from_portfolio(portfolio_id, stock_symbol):
    try:
        conn = sqlite3.connect('stock_portfolio.db')
        cursor = conn.cursor()

        cursor.execute("DELETE FROM stocks WHERE portfolio_id = ? AND stock_symbol = ?", (portfolio_id, stock_symbol))
        conn.commit()

        if cursor.rowcount > 0:
            print(f"股票 {stock_symbol} 移除成功")
        else:
            print(f"股票 {stock_symbol} 未找到")
    except sqlite3.Error as e:
        print(f"数据库错误: {e}")
    finally:
        conn.close()

# 显示所有股票组合
def display_portfolios():
    try:
        conn = sqlite3.connect('stock_portfolio.db')
        cursor = conn.cursor()

        cursor.execute("SELECT * FROM portfolios")
        portfolios = cursor.fetchall()

        for portfolio in portfolios:
            print(f"\n组合ID: {portfolio[0]}, 创建日期: {portfolio[1]}")
            cursor.execute("SELECT stock_symbol FROM stocks WHERE portfolio_id = ?", (portfolio[0],))
            stocks = cursor.fetchall()
            print("包含股票: " + ", ".join([stock[0] for stock in stocks]))
    except sqlite3.Error as e:
        print(f"数据库错误: {e}")
    finally:
        conn.close()

# 测试函数
def test_portfolio_management():
    # 假设你已经有了一个portfolio_id为1的股票组合
    portfolio_id = 1

    # 要测试的股票代码列表
    stock_symbols = ["AAPL", "MSFT", "AMZN", "GOOGL", "TSLA", "JPM", "JNJ", "KO", "BA", "NVDA"]

    # 添加股票到组合
    for symbol in stock_symbols:
        add_stock_to_portfolio(portfolio_id, symbol)

    # 显示所有组合
    display_portfolios()

    # 从组合中移除一个股票
    remove_stock_from_portfolio(portfolio_id, "TSLA")

    # 再次显示所有组合，检查'TSLA'是否被移除
    display_portfolios()

# 调用测试函数
test_portfolio_management()


无效的股票代码: AAPL
无效的股票代码: MSFT
无效的股票代码: AMZN
无效的股票代码: GOOGL
无效的股票代码: TSLA
无效的股票代码: JPM
无效的股票代码: JNJ
无效的股票代码: KO
无效的股票代码: BA
无效的股票代码: NVDA
数据库错误: no such table: portfolios
数据库错误: no such table: stocks
数据库错误: no such table: portfolios
